In [1]:
import re
import string
import pandas as pd
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import LinearSVC

In [ ]:
nlp = spacy.load("es_core_news_sm")

In [ ]:
def filtro_usuarios(content):
    texto = content
    indice_arroba = texto.find("@")
    while indice_arroba > -1:
        indice_espacio = texto.find(" ", indice_arroba)
        if indice_espacio == -1:
            texto = texto[:indice_arroba]
        else:
            texto = texto[:indice_arroba] + texto[indice_espacio:]
        indice_arroba = texto.find("@")
    return 

In [ ]:
for fin, fout in zip(fins, fouts):
    with open(fin, "rt") as fin_file:
        with open(fout, "wt") as fout_file:
            for line in fin_file:
                #Reemplazar los caracteres unicode, no se dejaron tildes porque causan error
                strtmp1 = line.replace('\\u00f1', 'ñ')
                strtmp1 = strtmp1.replace('\\u00e1', 'a')
                strtmp1 = strtmp1.replace('\\u00e9', 'e')
                strtmp1 = strtmp1.replace('\\u00ed', 'i')
                strtmp1 = strtmp1.replace('\\u00f3', 'o')
                strtmp1 = strtmp1.replace('\\u00fa', 'u')
                strtmp1 = strtmp1.replace('\\u00bf', '¿')
                strtmp1 = strtmp1.replace('\\u00a1', '¡')
                strtmp1 = strtmp1.replace('\\u00d1', 'Ñ')
                strtmp1 = strtmp1.replace('\\u00c1', 'A')
                strtmp1 = strtmp1.replace('\\u00c9', 'E')
                strtmp1 = strtmp1.replace('\\u00cd', 'I')
                strtmp1 = strtmp1.replace('\\u00d3', 'O')
                strtmp1 = strtmp1.replace('\\u00da', 'U')
                strtmp1 = strtmp1.replace('\\u00fc', 'ü')
                strtmp1 = strtmp1.replace('\\u00b0', '')
                #Quitar el inicio y el fin del json para dejar solo los tweets
                strtmp1 = strtmp1.replace('{"tweets": {"tweet": ', '')
                strtmp1 = strtmp1.replace(']}}', ']')
                #Quitar el diccionario que contiene la polaridad y dejarla solo con su valor de sentimiento
                strtmp1 = strtmp1.replace('"sentiment": {"polarity": {"value": ', '"sentiment": ')
                strtmp1 = strtmp1.replace('"NONE"}}', '"NONE"')
                #Asignamos al sentimiento positivo el valor de 1
                strtmp1 = strtmp1.replace('"P"}}', '1')
                strtmp1 = strtmp1.replace('"NEU"}}', '"NEU"')
                #Asignamos al sentimiento negativo el valor de 0
                strtmp1 = strtmp1.replace('"N"}}', '0')
                #eliminación de puntuaciones
                strtmp1 = re.sub('[¡!#$).;¿?&°]', '', strtmp1.lower())
                fout_file.write(strtmp1)

In [ ]:
puntuaciones = string.punctuation + "¡¿"
stopwords = set(STOP_WORDS)

def limpieza_de_datos(oracion):
    # Eliminar emojis utilizando expresiones regulares
    oracion = re.sub(r'[^\w\s,]', '', oracion)
    
    doc = nlp(oracion)
    
    tokens_limpios = []
    for token in doc:
        # Ignorar tokens de longitud uno y caracteres no alfabéticos
        if token.is_alpha and len(token.text) > 1:
            # Convertir a minúsculas y lematizar sustantivos y adjetivos
            if token.lemma_ != "-PRON-" and token.pos_ in ['NOUN', 'ADJ']:
                temp = token.lemma_.lower().strip()
            else:
                temp = token.lower_
            
            # Filtrar stopwords y puntuaciones
            if temp not in stopwords and temp not in puntuaciones:
                tokens_limpios.append(temp)
    
    return tokens_limpios